In [35]:
# !pip install datasets
!pip install --upgrade pip
!pip install --upgrade transformers
!pip install torch
!pip install transformers[torch]
!pip install 'accelerate>=0.26.0'
!pip install evaluate
!pip install scikit-learn
!pip install seqeval



# Import necessary libraries
from transformers import BertTokenizer, BertConfig, BertForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, PreTrainedTokenizerFast
from datasets import load_dataset, load_from_disk, DatasetDict
import os
import torch
import numpy as np
import pickle

# Disable W&B logging if not needed
os.environ["WANDB_DISABLED"] = "true"  # Uncomment to disable W&B logging


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# create subset
dataset = load_dataset("text", data_files=["dataset/bookcorpus_subset_2gb_new.txt", "dataset/wikipedia_subset_7gb.txt"], sample_by="paragraph")
print(dataset)

# Take a 1% subset 
subset_dataset = dataset['train'].train_test_split(test_size=0.01, seed=42)['test']
# Save the subset to a new file 
subset_dataset.save_to_disk('dataset/subset_dataset')

In [41]:
# Load tokenizer
tokenizer_file = "tokenizers/wordpiece-tokenizer.json"
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'mask_token': '[MASK]', 'unk_token': '[UNK]'})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# Load subset
subset_dataset = load_from_disk("dataset/subset_dataset")
# Tokenize subset
tokenized_dataset = subset_dataset.map(tokenize_function, batched=True)
# Save for later
tokenized_dataset.save_to_disk("tokenized_datasets_wordpiece")

Saving the dataset (1/1 shards): 100%|██████████| 500818/500818 [00:00<00:00, 940471.65 examples/s]


In [42]:
# View Tokenized dataset info
tokenized_dataset

Dataset({
    features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 500818
})

In [43]:
# Split the dataset into 80% training and 20% testing
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
# Split the 20% testing set into 50% validation and 50% test sets
val_test_split = train_test_split['test'].train_test_split(test_size=0.5)
# Create a dataset dictionary
tokenized_datasets = DatasetDict({
    'train': train_test_split['train'],
    'validation': val_test_split['train'],
    'test': val_test_split['test']
})

In [44]:
# View dataset split info
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 400654
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50082
    })
    test: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50082
    })
})

In [45]:
# Train BASE BERT model with the tokenizer

# Define model configuration
config = BertConfig(
    vocab_size=30000,
    hidden_size=128,         # Reduced hidden size
    num_hidden_layers=2,     # Only 2 transformer layers
    num_attention_heads=4,   # Adjusted to fit with hidden size
    intermediate_size=512,   # Feed-forward layer size
    max_position_embeddings=128,  # Smaller max sequence length if needed
    type_vocab_size=2,
    initializer_range=0.02,
    layer_norm_eps=1e-12
)

# Initialize the model
model = BertForMaskedLM(config)

# Resize token embeddings
model.resize_token_embeddings(len(tokenizer))

# Set up training arguments
training_args = TrainingArguments(
    output_dir = './results',
    evaluation_strategy='epoch',
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    save_strategy='no',
    logging_strategy='epoch'
)

# Create a data collator 
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

# Train the model
trainer.train()

# Save Model
model.save_pretrained("Wordpiece_BERT_Base")

/home/elicer/.local/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,3.040100,2.542329
2,2.266100,1.855856
3,1.957600,1.736889


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

